In [11]:
import torch.utils.data as data
import torch.nn as nn

In [12]:
def train_process(training_dataset):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(device)
    training_loader = data.DataLoader(training_dataset, batch_size=32, shuffle=True)
    model = Word2VecModel(len(training_dataset.word2idx), 256)
    model = model.to(device)
    model.train()
    model.zero_grad()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.00001)
    loss_function = nn.CrossEntropyLoss()
    
    #for data in training_loader:

%run data_preprocess.ipynb
%run utils.ipynb
%run modules.ipynb
training_set = read_data("./datasets/trainset.csv")
training_dataset= Word2VecDataset(training_set, 2)
train_process(training_dataset)

Reading data...
Makeing dictionary...
Generating pair word data...


TypeError: to() received an invalid combination of arguments - got (device), but expected one of:
 * (torch.device device, torch.dtype dtype, bool non_blocking, bool copy, *, torch.memory_format memory_format)
 * (torch.dtype dtype, bool non_blocking, bool copy, *, torch.memory_format memory_format)
 * (Tensor tensor, bool non_blocking, bool copy, *, torch.memory_format memory_format)


In [ ]:
def main():
    training_set = read_data("./datasets/trainset.csv")
    train_process(training_set)